In [ ]:
import os
import sys

import h5py

"""
このnotebookがbaseフォルダを読めるようになるための処理
"""

root_rel = '../../'

# 相対パスを絶対パスに変換してsys.pathに追加
root_abs = os.path.abspath(root_rel)
if root_abs not in sys.path:
    sys.path.append(root_abs)
print(root_abs)


In [ ]:
# 使用データフォルダまでのpath設定
data_root_path = "/Volumes/T7 Touch/data"  # 人によって書き換え
# data_root_path = "/Volumes/Ohta_ssd/data" # 太田研共有SSDのMacの場合のpath

# 保存データ(図や動画)のpath設定
save_root_path = root_abs + "/save/"

# Folderインスタンスを作成
# Folderクラスに検索や表示の関数を入れてる
import model.File.Read as Read
import model.File.Save as Save

read = Read.Read(data_root_path=data_root_path)
save = Save.Save(save_root_path=save_root_path)


In [ ]:
# Run設定
# runまで選択
sample_num = 0
experiment_num = 2
run_num = 0

read.select_data_folder(sample_num=sample_num,
                        experiment_num=experiment_num,
                        run_num=run_num)

print(f"データのpath: {read.data_path}\n")
save.set_run_save_dir(run_name=read.run_name)

# 使用データpathの設定
read.set_filepaths()


In [ ]:
# LAMBDAのmaskは固定
mask_path = '/Users/ishizawaosamu/work/ipynb/BL10XU_notebook/base/data/lambda_mask.npy'

In [ ]:
path_vars = [
    # T
    'spe_path',
    'calib_path',
    'dist_path',
    # XRD
    'nxs_path',
    'poni_path',
    'mask_path'
]

path_dict = {
    path_vars[0]: read.spe_path,
    path_vars[1]: read.calib_path,
    path_vars[2]: read.dist_path,
    path_vars[3]: read.nxs_path,
    path_vars[4]: read.poni_path,
    path_vars[5]: mask_path
}

In [ ]:
# HDF5ファイルを作成する
"""
    *** 一応ファイルがある場合は実行されないようにしているが、もし実行された場合は初期化されるので注意
"""
hdf5_path = f"/Volumes/T7 Touch/data/HDF5/{read.run_name}.hdf5"

path_layer = 'entry/path/'

if not os.path.exists(hdf5_path):
    with h5py.File(hdf5_path, 'w') as f:
        for path_var in path_vars:
            path = path_layer + path_var
            f.create_dataset(path, data=path_dict[path_var])

In [ ]:
from model.File.HDF5 import HDF5

In [ ]:
with h5py.File(hdf5_path, 'r') as f:
    hdf = HDF5(f=f)

In [ ]:
hdf.path_list

In [ ]:
with h5py.File(hdf5_path, 'r') as f:
    hdf.show_all_hierarchy(f=f, display_length=80)

In [ ]:
# 辞書型の方が拡張するとき便利そうなのでjsonにする
# 書き加える場合は、一度読み込んで辞書を編集してdumpしなおす
import json

run = {"hdf5_path": hdf5_path}
with open('setting.json', 'w') as file:
    json.dump(run, file)
